<a href="https://colab.research.google.com/github/blurry-mood/Recommendation-System/blob/main/notebooks/MovieLens_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Run cells bellow to download and preprocess the **MovieLens20M** dataset.

Estimated time is around 45 minutes.

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-20m.zip

--2022-01-03 12:40:58--  https://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  28.2MB/s    in 7.4s    

2022-01-03 12:41:06 (25.5 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]



In [ ]:
# %cd /content/drive/MyDrive/MovieLens

/content/drive/MyDrive/MovieLens


In [ ]:
!unzip ml-20m.zip

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


All Items

In [ ]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
m = pd.read_csv('ml-20m/movies.csv')  # Reading movies ??
m = m[m.movieId.notnull()].reindex()
m['itemid']=m.movieId.progress_map(lambda x: str(int(x)))
m['product_name'] = m['title']
items = m[['itemid','product_name','genres']]
items.to_json('items.json')

  0%|          | 0/27278 [00:00<?, ?it/s]

In [ ]:
items[items.itemid == "4993"]

,itemid,product_name,genres
4897,4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy


In [ ]:
items.to_csv('items.csv')

Purchases and grouped purchases.

In [ ]:
interactions = pd.read_csv('ml-20m/ratings.csv')
interactions = interactions[interactions.rating>=4.]
print(interactions.shape)
interactions = interactions.sort_values(['userId','timestamp']) 
interactions['itemid'] = interactions['movieId'].progress_map(str)
interactions['userid'] = interactions['userId'].progress_map(str)
interactions['amount'] = 1
interactions['date'] = interactions['timestamp']
interactions[['itemid','userid','amount','date']]
interactions.to_json("purchases.json")
interactions['itemids'] = interactions[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = interactions[['userId','itemids']].drop_duplicates()
iii.to_json('purchases_txt.json')

(9995410, 4)


  0%|          | 0/9995410 [00:00<?, ?it/s]

  0%|          | 0/9995410 [00:00<?, ?it/s]

In [ ]:
interactions

Keep only users with 5 or more interactions.

Purchases.

In [ ]:
purchases=pd.read_json('purchases.json')
purchases['userid'] = purchases.userid.apply(str)
purchases['itemid'] = purchases.itemid.apply(str)
purchases_item_counts = purchases[['userid','itemid']]
purchases_user_counts = purchases[['userid','itemid']]
purchases_user_count = purchases.groupby(['userid']).size().to_frame('nr_of_purchases').reset_index()
purchases_user_count = purchases_user_count.sort_values(by=['nr_of_purchases'], ascending=False)
pu5=purchases_user_count[purchases_user_count.nr_of_purchases>=5]
purchases_pu5 = purchases[purchases.userid.isin(pu5.userid)]
purchases_item_count_pu5 = purchases_pu5.groupby(['itemid']).size().to_frame('nr_of_purchases').reset_index()
purchases_item_count_pu5 = purchases_item_count_pu5.sort_values(by=['nr_of_purchases'], ascending=False)
purchases_pu5.to_json('purchases_pu5.json')

In [ ]:
purchases_pu5

Grouped purchases.

In [ ]:
purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = purchases_pu5[['userId','itemids']].drop_duplicates()
iii['userid']=iii['userId'].apply(str)
iii = iii[['userid','itemids']]
iii.to_json('purchases_txt_pu5.json')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Users.

In [ ]:
iii['userid'].to_frame().to_json('users_pu5.json')

Items

In [ ]:
items[items.itemid.isin(purchases_item_count_pu5.itemid)].to_json("items_pu5.json")

In [ ]:
items[items.itemid.isin(purchases_item_count_pu5.itemid)]

,itemid,product_name,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27271,131250,No More School (2000),Comedy
27272,131252,Forklift Driver Klaus: The First Day on the Jo...,Comedy|Horror
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy


Items sorted by number of interactions

In [ ]:
purchases_item_count_pu5.to_json("items_sorted_pu5.json")

Users sorted by number of interactions

In [ ]:
pu5.to_json("users_sorted_pu5.json")

Create train, val and test split.

In [ ]:
users = pd.read_json('users_pu5.json')
shuffled_users = users.sample(frac=1., random_state=42)
test_users = shuffled_users.iloc[:10000]
val_users = shuffled_users.iloc[10000:20000]
train_users = shuffled_users.iloc[20000:]

test_users.to_json("test_users.json")
val_users.to_json("val_users.json")
train_users.to_json("train_users.json") 

len(train_users),len(val_users),len(test_users)

(116677, 10000, 10000)

In [ ]:
! cp *json /content/drive/MyDrive/MovieLens/

List of generated json files with preprocessed data.

In [ ]:
!ls *.json

items.json	       purchases_pu5.json      train_users.json
items_pu5.json	       purchases_txt.json      users_pu5.json
items_sorted_pu5.json  purchases_txt_pu5.json  users_sorted_pu5.json
purchases.json	       test_users.json	       val_users.json


In [ ]:
import tensorflow as tf
toki = tf.keras.preprocessing.text.Tokenizer()

In [ ]:
toki.fit_on_texts(purchases_item_count_pu5.itemid.to_list())
_, num_words = toki.texts_to_matrix(['xx']).shape

In [ ]:
purchases_item_count_pu5.shape

(14408, 2)

In [ ]:
toki.texts_to_matrix(["318,296"])

array([[0., 1., 1., ..., 0., 0., 0.]])

In [ ]:
purchases_item_count_pu5

,itemid,nr_of_purchases
4242,318,29439
3866,296,26067
8848,593,21510
7694,527,21328
2868,260,21262
...,...,...
801,111732,1
799,111680,1
6475,45611,1
796,111630,1


In [ ]:
import tensorflow as tf
import pandas as pd


title_list = pd.read_json('items_pu5.json').product_name.values.tolist()


toki = tf.keras.preprocessing.text.Tokenizer()
toki.fit_on_texts(title_list)

def vasp_model(fav_movies):
	token = toki.texts_to_matrix(fav_movies)
	_, num_words = token.shape
	return num_words

In [ ]:
title_list[0]

'Toy Story (1995)'

In [ ]:
vasp_model(["Ok"])

13918

In [ ]:
# purchases_item_count_pu5.itemid.to_list()